# Medical-ML

## Introduction

This project aims to explore and analyze heart failure data to develop and evaluate machine learning models capable of delivering accurate predictions.

The dataset analyzed in this project focuses on heart failure, specifically determining whether a patient has heart disease based on various demographic characteristics and biomarkers. Originally titled *Heart Failure Prediction*, the dataset was sourced from [Kaggle](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?select=heart.csv).

**The introductory part contains setting up the environment, and function definitions.**

In [ ]:
import dalex as dx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import warnings

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline
from scipy.stats import spearmanr
from scipy.stats.contingency import association
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
    TunedThresholdClassifierCV,
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
np.random.seed(13)
sns.set_theme(style='whitegrid')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# An improvement upon KNNImputer found in sklearn.
class BetterKNNImputer(KNNImputer):
    def __init__(
        self,
        numeric_cols: list,
        categorical_cols: list,
        target_col: str,
        is_target_numeric=True,
    ):
        self.numeric_cols = [col for col in numeric_cols if col != target_col]
        self.categorical_cols = [col for col in categorical_cols if col != target_col]
        self.target_col = target_col
        self.is_target_numeric = is_target_numeric
        
        self.numeric_transformer = StandardScaler()
        self.categorical_transformer = OneHotEncoder(handle_unknown='ignore')
        
        if is_target_numeric:
            self.target_transformer = StandardScaler()
            self.undersampler = None
        else:
            self.target_transformer = OneHotEncoder(handle_unknown='ignore')
            # NearMiss-1 seems to do well here.
            self.undersampler = NearMiss()
            
        super().__init__()
        self.weights = 'distance'

    def _preprocess_data_fit(
        self,
        X: pd.DataFrame,
    ) -> tuple[pd.DataFrame, pd.DataFrame, np.ndarray, np.ndarray]:
        # Extract the column of interest from X.
        target = X[[self.target_col]].copy()
        X = X.drop(self.target_col, axis=1)
        
        # Standardize numeric columns.
        X[self.numeric_cols] = self.numeric_transformer.fit_transform(
            X[self.numeric_cols]
        )
        
        # Perform One Hot Encoding on categorical variables.
        categorical_dummies = self.categorical_transformer.fit_transform(
            X[self.categorical_cols]
        )
        categorical_features = self.categorical_transformer.get_feature_names_out()
        categorical_dummies = pd.DataFrame(
            data=categorical_dummies.toarray(),
            columns=categorical_features,
        )
        
        categorical_features = [
            col for col in categorical_features if not col.endswith('nan')
        ]
        categorical_dummies = categorical_dummies[categorical_features]
        
        # Concat X so everything in it is encoded.
        X = X.drop(self.categorical_cols, axis=1)
        X = pd.concat([X.reset_index(drop=True), categorical_dummies], axis=1)

        # Perform undersampling if target variable is categorical.
        X_mask = ~X.isna().any(axis=1).to_numpy()
        old_target_mask = ~target.isna().any(axis=1).to_numpy()
        X_target_mask = X_mask & old_target_mask
        if not self.is_target_numeric:
            X_undersampled, target_undersampled = self.undersampler.fit_resample(
                X[X_target_mask],
                target[X_target_mask],
            )
            X = pd.concat([X[~X_target_mask], X_undersampled])
            target = pd.concat([target[~X_target_mask], target_undersampled])

        # Make sure index for X isn't nonsense.
        X = X.reset_index(drop=True)

        # Mask is useful for fitting.
        new_target_mask = ~target.isna().any(axis=1).to_numpy()
        # Transform the target variable.
        target = self.target_transformer.fit_transform(target)
        
        return X, target, old_target_mask, new_target_mask

    def _preprocess_data_transform(
        self,
        X: pd.DataFrame,
    ) -> tuple[pd.DataFrame, pd.DataFrame, np.ndarray]:
        # Extract the column of interest from X.
        target = X[[self.target_col]].copy()
        X = X.drop(self.target_col, axis=1)

        # Standardize numeric columns.
        X[self.numeric_cols] = self.numeric_transformer.transform(
            X[self.numeric_cols]
        )

        # Perform One Hot Encoding on categorical variables.
        categorical_dummies = self.categorical_transformer.transform(
            X[self.categorical_cols]
        )
        categorical_features = self.categorical_transformer.get_feature_names_out()
        categorical_dummies = pd.DataFrame(
            data=categorical_dummies.toarray(),
            columns=categorical_features,
        )
        categorical_features = [
            col for col in categorical_features if not col.endswith('nan')
        ]
        categorical_dummies = categorical_dummies[categorical_features]

        # Concat X so everything in it is encoded.
        X = X.drop(self.categorical_cols, axis=1)
        X = pd.concat([X.reset_index(drop=True), categorical_dummies], axis=1)

        # Mask is useful for transforming.
        target_mask = ~target.isna().any(axis=1).to_numpy()
        # Transform the target variable.
        target = self.target_transformer.transform(target)
        
        return X, target, target_mask

    def fit(self, X: pd.DataFrame):
        new_data = X.copy()
        X, target, old_target_mask, new_target_mask = self._preprocess_data_fit(X)

        if self.is_target_numeric:
            target = pd.DataFrame(data=target, columns=[self.target_col])
            X = pd.concat([X, target], axis=1)
        else:
            target_features = self.target_transformer.get_feature_names_out()
            target = pd.DataFrame(data=target.toarray(),columns=target_features)
            target_features = [
                col for col in target_features if not col.endswith('nan')
            ]
            target = target[target_features]
            # Makes it so missing columns are actually read as such.
            target[~new_target_mask] = np.nan
            X = pd.concat([X, target], axis=1)

        # Set n_neighbors using a heuristic.
        self.n_neighbors = int(len(target) ** 0.5)
        
        return super().fit(X)

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        new_data = X.copy()
        X, target, target_mask = self._preprocess_data_transform(X)

        if self.is_target_numeric:
            target = pd.DataFrame(data=target, columns=[self.target_col])
            X = pd.concat([X, target], axis=1)
            new_target = super().transform(X)[:, [-1]]
            new_target = self.target_transformer.inverse_transform(new_target)[:, 0]
            new_data[self.target_col] = new_target
        else:
            target_features = self.target_transformer.get_feature_names_out()
            target = pd.DataFrame(data=target.toarray(), columns=target_features)
            target_features = [
                col for col in target_features if not col.endswith('nan')
            ]
            target = target[target_features]
            # Makes it so missing columns are actually read as such.
            target[~target_mask] = np.nan
            
            X = pd.concat([X, target], axis=1)
            new_target = super().transform(X)[:, (-len(target_features)):]
            new_target = self.target_transformer.categories_[0][
            np.argmax(new_target, axis=1)
            ]
            new_data[self.target_col] = new_target
        
        return new_data

def cramerv(var1: np.ndarray, var2: np.ndarray) -> float:
    crosstab = np.array(pd.crosstab(var1,var2, rownames=None, colnames=None))
    return association(crosstab)

def diagnostic_odds_ratio(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    tn, fp, fn, tp = confusion_matrix(ground_truth, predictions).ravel()
    if fp == 0 and fn == 0:
        return tp * tn
    elif fp == 0 or fn == 0:
        return 0
    else:
        return (tp * tn) / (fp * fn)

diagnostic_odds_ratio_scorer = make_scorer(
    diagnostic_odds_ratio,
    greater_is_better=True,
)

## EDA

The **Exploratory Data Analysis** seeks to uncover the underlying structure of the data, verify its suitability for machine learning, and ensure there are no missing values, inbalances, or structural issues.

### Summary Statistics

**The head of data is presented below.**

In [ ]:
data = pd.read_csv('heart_failure.csv')
data.head()

In [ ]:
data['FastingBS'] = data['FastingBS'].astype(str)
numeric_cols = [
    'Age',
    'RestingBP',
    'Cholesterol',
    'MaxHR',
    'Oldpeak',
]
categorical_cols = [
    'Sex',
    'ChestPainType',
    'FastingBS',
    'RestingECG',
    'ExerciseAngina',
    'ST_Slope',
]

**Verifying if there are missing values.**

In [ ]:
display(pd.DataFrame(data.isna().sum(), columns = ['missing']))

**There aren't any missing values in the data.** 

It's useful to review summary statistics for numeric columns to identify any unusual or illogical values. Those values may not only be outliers, but, in extreme cases, could indicate errors in the dataset (e.g., an age of 167).

In [ ]:
data[numeric_cols].describe()

Only *Age* and *MaxHR* seem to have reasonable values. Other variables seem to have a few potential outliers - thus, boxplots will be used to check for any isolated extremes.

### Distributions of numeric variables (boxplots)

In [ ]:
ax = sns.boxplot(data['Age'])
_ = ax.set(title='Box Plot of Age', ylabel=None)

In [ ]:
ax = sns.boxplot(data['RestingBP'])
_ = ax.set(title='Box Plot of Resting Blood Pressure', ylabel=None)

It seems there are a few extreme values for *Resting Blood Pressure*. **However, these are likely not a result of a measurement error, but a fat-tailed distribution, and should be kept in the dataset.**

A resting blood pressure of 80mmHg is low and could indicate hypotension, whereas 200mmHg  signals severe hypertension (it is extremely high but possible to attain).

In [ ]:
ax = sns.boxplot(data['Cholesterol'])
_ = ax.set(title='Box Plot of Cholesterol', ylabel=None)

**For *Cholesterol* a few observations should be removed or their value should be imputed.** In particular, a human cannot have 0 cholesterol, and the value above 500 warrants removal - such a case is extremely rare.

In [ ]:
ax = sns.boxplot(data['MaxHR'])
_ = ax.set(title='Box Plot of Max Heart Rate', ylabel=None)

There are a few extreme values for *Max Heart Rate*. **However, these are likely not a result of a measurement error and should be kept in the dataset.**

A maximum heart rate below 80 is highly unusual; however it is possible and may indicate severe cardiovascular issues.

In [ ]:
ax = sns.boxplot(data['Oldpeak'])
_ = ax.set(title='Box Plot of ST Depression', ylabel=None)

There are a few outliers for *ST Depression*; however, **they remain within the realistic range** - a severe depression can oscillate around 5 mm and may indicate myocardial ischemia or other cardiac issues.

For *Cholesterol* observations above 500 will be removed, and observations with 0 will be replaced through *KNN Inputer*.

In [ ]:
data = data[data['Cholesterol'] < 500]
data.loc[data['Cholesterol'] == 0, 'Cholesterol'] = np.nan

### Relationships between numeric variables

**Now collinearity will be checked.**

In [ ]:
data[numeric_cols].corr()

There doesn't seem to be a problem with collinearity, as all the values are within a proper range.

**Now the monotonic relationship between the numeric variables and the response variable will be checked via Spearman's $\rho$.**

In [ ]:
numeric_predictive_power = pd.DataFrame(columns = ['rho'])
for col in numeric_cols:
    temp_data = data[~data[col].isna()]
    numeric_predictive_power.loc[col, :] = (
        [spearmanr(temp_data[col], temp_data['HeartDisease']).statistic]
    )
numeric_predictive_power

**It seems that *RestingBP* and *Cholesterol* may not have that much of an effect on the risk of heart disease**, which is surprising, considering that cholesterol is commonly associated with it. However, a low correlation does not imply that the variables are not significant once models are built.

### Relationships between categorical variables

Now categorical variables will be assessed. First, it will be verified if there are any extremely rare categories (< 1%).

In [ ]:
for col in categorical_cols:
    display(pd.DataFrame(data[col].value_counts(normalize=True)))

The proportions fall within an acceptable range.

**The dependency between categorical variables and the response variable will be checked using *Cramer's V*.**

In [ ]:
categorical_predictive_power = pd.DataFrame(columns = ['cramer_V'])
for col in categorical_cols:
    categorical_predictive_power.loc[col, :] = (
        [cramerv(data[col], data['HeartDisease'])]
    )
categorical_predictive_power

Most of these variables seem to have some predictive power, but a possible exception is *RestingECG*. 

**The class imbalance in the response variable should also be checked.**

In [ ]:
print(
    'Proportion of people diagnosed with heart disease:',
    data['HeartDisease'].sum() / len(data)
)

While the imbalance isn't extreme, it's worth considering during model training. 

## Machine Learning

### Data Preparation

In [ ]:
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.3)

In [ ]:
imputer = BetterKNNImputer(
    numeric_cols,
    categorical_cols,
    'Cholesterol',
    is_target_numeric=True,
)
X_train['Cholesterol'] = imputer.fit_transform(X_train)['Cholesterol']
X_test['Cholesterol'] = imputer.transform(X_test)['Cholesterol']

In [ ]:
X_train[numeric_cols].describe()

In [ ]:
X_test[numeric_cols].describe()

In [ ]:
for col in categorical_cols:
    display(pd.DataFrame(X_train[col].value_counts(normalize=True)))
    display(pd.DataFrame(X_test[col].value_counts(normalize=True)))

In [ ]:
print('Train proportion:', y_train.sum() / len(y_train))
print('Test proportion:', y_test.sum() / len(y_test))

Split seems fairly good. It's time to try building the some models. We're going to use **AP (Average Precision, threshold-agnostic metric used for tuning hiperparameters)** and **DOR (Diagnostic Odds Ratio, used for tuning the threshold and SVM)** for cross-validation.  We will test a few potential models:
- logistic regression,
- random forest,
- decision tree boosting,
- SVM.

First we will simply use the dataset without any modifications, then apply observation weighting, and afterwards we will also attempt to use undersampling/oversampling.

In [ ]:
valid_thresholds = np.linspace(0.01, 0.5, 201)

### No Weighting or Under-/Oversampling

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
    ]
)
logistic_pipeline = Pipeline(
    [
        ('transformer', transformer),
        ('model', LogisticRegression(penalty=None)),
    ]
)

threshold_tuning = TunedThresholdClassifierCV(
    logistic_pipeline,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
logistic_pipeline = threshold_tuning.estimator_
logistic_threshold = threshold_tuning.best_threshold_
display(logistic_pipeline)
print('Best threshold:', logistic_threshold)

In [ ]:
transformer = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
forest_pipeline = Pipeline(
    [
        ('transformer', transformer),
        ('model', RandomForestClassifier()),
    ]
)

parameters = {
    'model__n_estimators': [100, 150, 200, 250],
    'model__criterion': ['gini', 'entropy'],
    'model__max_features': [0.1, 0.2, 0.3, 0.4],
    'model__max_depth': [2, 3, 5],
}
parameter_tuning = RandomizedSearchCV(
    forest_pipeline,
    parameters,
    n_iter=50,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
forest_pipeline = parameter_tuning.best_estimator_
display(forest_pipeline)

threshold_tuning = TunedThresholdClassifierCV(
    forest_pipeline,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
forest_threshold = threshold_tuning.best_threshold_
print('Best threshold:', forest_threshold)

In [ ]:
transformer = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

boost_pipeline = Pipeline(
    [
        ('transformer', transformer),
        ('model', AdaBoostClassifier(algorithm='SAMME')),
    ]
)

parameters = {
    'model__n_estimators': [5, 10, 15, 20],
    'model__learning_rate': [0.4, 0.8, 1.2, 1.6, 2],
    'model__estimator': [
        DecisionTreeClassifier(
            max_depth=1,
            criterion=crit)
        for crit in ['gini', 'entropy']
    ],
}
parameter_tuning = GridSearchCV(
    boost_pipeline,
    parameters,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
boost_pipeline = parameter_tuning.best_estimator_
display(boost_pipeline)

threshold_tuning = TunedThresholdClassifierCV(
    boost_pipeline,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
boost_threshold = threshold_tuning.best_threshold_
print('Best threshold:', boost_threshold)

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ]
)

svm_pipeline = Pipeline(
    [
        ('transformer', transformer),
        ('model', SVC()),
    ]
)

parameters = [
    {
        'model__kernel': ['linear'],
        'model__C': [2 ** (i - 9) for i in range(19)],
    },
    {
        'model__kernel': ['poly'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__degree': [i + 2 for i in range(4)],
        'model__coef0': [0, 0.1, 0.5, 1, 2, 10],
    },
    {
        'model__kernel': ['rbf'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__gamma': [2 ** (i - 8) for i in range(17)],
    },
]
parameter_tuning = RandomizedSearchCV(
    svm_pipeline,
    parameters,
    n_iter=50,
    scoring=diagnostic_odds_ratio_scorer,
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
svm_pipeline = parameter_tuning.best_estimator_
display(svm_pipeline)

### Weighting

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
    ]
)
logistic_pipeline_weighted = Pipeline(
    [
        ('transformer', transformer),
        ('model', LogisticRegression(penalty=None, class_weight='balanced')),
    ]
)

threshold_tuning = TunedThresholdClassifierCV(
    logistic_pipeline_weighted,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
logistic_pipeline_weighted = threshold_tuning.estimator_
logistic_threshold_weighted = threshold_tuning.best_threshold_
display(logistic_pipeline_weighted)
print('Best threshold:', logistic_threshold_weighted)

In [ ]:
transformer = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
forest_pipeline_weighted = Pipeline(
    [
        ('transformer', transformer),
        ('model', RandomForestClassifier(class_weight='balanced')),
    ]
)

parameters = {
    'model__n_estimators': [100, 150, 200, 250],
    'model__criterion': ['gini', 'entropy'],
    'model__max_features': [0.1, 0.2, 0.3, 0.4],
    'model__max_depth': [2, 3, 5],
}
parameter_tuning = RandomizedSearchCV(
    forest_pipeline_weighted,
    parameters,
    n_iter=50,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
forest_pipeline_weighted = parameter_tuning.best_estimator_
display(forest_pipeline_weighted)

threshold_tuning = TunedThresholdClassifierCV(
    forest_pipeline_weighted,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
forest_threshold_weighted = threshold_tuning.best_threshold_
print('Best threshold:', forest_threshold_weighted)

In [ ]:
transformer = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

boost_pipeline_weighted = Pipeline(
    [
        ('transformer', transformer),
        ('model', AdaBoostClassifier(algorithm='SAMME')),
    ]
)

parameters = {
    'model__n_estimators': [5, 10, 15, 20],
    'model__learning_rate': [0.4, 0.8, 1.2, 1.6, 2],
    'model__estimator': [
        DecisionTreeClassifier(
            max_depth=1,
            criterion=crit,
            class_weight='balanced')
        for crit in ['gini', 'entropy']
    ],
}
parameter_tuning = GridSearchCV(
    boost_pipeline_weighted,
    parameters,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
boost_pipeline_weighted = parameter_tuning.best_estimator_
display(boost_pipeline_weighted)

threshold_tuning = TunedThresholdClassifierCV(
    boost_pipeline_weighted,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
boost_threshold_weighted = threshold_tuning.best_threshold_
print('Best threshold:', boost_threshold_weighted)

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ]
)

svm_pipeline_weighted = Pipeline(
    [
        ('transformer', transformer),
        ('model', SVC(class_weight='balanced')),
    ]
)

parameters = [
    {
        'model__kernel': ['linear'],
        'model__C': [2 ** (i - 9) for i in range(19)],
    },
    {
        'model__kernel': ['poly'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__degree': [i + 2 for i in range(4)],
        'model__coef0': [0, 0.1, 0.5, 1, 2, 10],
    },
    {
        'model__kernel': ['rbf'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__gamma': [2 ** (i - 8) for i in range(17)],
    },
]
parameter_tuning = RandomizedSearchCV(
    svm_pipeline_weighted,
    parameters,
    n_iter=50,
    scoring=diagnostic_odds_ratio_scorer,
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
svm_pipeline_weighted = parameter_tuning.best_estimator_
display(svm_pipeline_weighted)

### Undersampling
We will use NearMiss undersampling. Each of the three variants will be tested during cross-validation.

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
    ]
)
logistic_pipeline_undersampled = Pipeline(
    [
        ('transformer', transformer),
        ('undersampler', NearMiss()),
        ('model', LogisticRegression(penalty=None)),
    ]
)

parameters = {'undersampler__version': [1, 2, 3]}
parameter_tuning = GridSearchCV(
    logistic_pipeline_undersampled,
    parameters,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
logistic_pipeline_undersampled = parameter_tuning.best_estimator_
display(logistic_pipeline_undersampled)

threshold_tuning = TunedThresholdClassifierCV(
    logistic_pipeline_undersampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
logistic_threshold_undersampled = threshold_tuning.best_threshold_
print('Best threshold:', logistic_threshold_undersampled)

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ]
)
forest_pipeline_undersampled = Pipeline(
    [
        ('transformer', transformer),
        ('undersampler', NearMiss()),
        ('model', RandomForestClassifier()),
    ]
)

parameters = {
    'undersampler__version': [1, 2, 3],
    'model__n_estimators': [100, 150, 200, 250],
    'model__criterion': ['gini', 'entropy'],
    'model__max_features': [0.1, 0.2, 0.3, 0.4],
    'model__max_depth': [2, 3, 5],
}
parameter_tuning = RandomizedSearchCV(
    forest_pipeline_undersampled,
    parameters,
    n_iter=50,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
forest_pipeline_undersampled = parameter_tuning.best_estimator_
display(forest_pipeline_undersampled)

threshold_tuning = TunedThresholdClassifierCV(
    forest_pipeline_undersampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
forest_threshold_undersampled = threshold_tuning.best_threshold_
print('Best threshold:', forest_threshold_undersampled)

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ]
)

boost_pipeline_undersampled = Pipeline(
    [
        ('transformer', transformer),
        ('undersampler', NearMiss()),
        ('model', AdaBoostClassifier(algorithm='SAMME')),
    ]
)
parameters = {
    'undersampler__version': [1, 2, 3],
    'model__n_estimators': [5, 10, 15, 20],
    'model__learning_rate': [0.4, 0.8, 1.2, 1.6, 2],
    'model__estimator': [
        DecisionTreeClassifier(
            max_depth=1,
            criterion=crit)
        for crit in ['gini', 'entropy']
    ],
}
parameter_tuning = RandomizedSearchCV(
    boost_pipeline_undersampled,
    parameters,
    n_iter=50,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
boost_pipeline_undersampled = parameter_tuning.best_estimator_
display(boost_pipeline_undersampled)

threshold_tuning = TunedThresholdClassifierCV(
    boost_pipeline_undersampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
boost_threshold_undersampled = threshold_tuning.best_threshold_
print('Best threshold:', boost_threshold_undersampled)

In [ ]:
transformer = ColumnTransformer(
    [
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ]
)

svm_pipeline_undersampled = Pipeline(
    [
        ('transformer', transformer),
        ('undersampler', NearMiss()),
        ('model', SVC()),
    ]
)

parameters = [
    {
        'undersampler__version': [1, 2, 3],
        'model__kernel': ['linear'],
        'model__C': [2 ** (i - 9) for i in range(19)],
    },
    {
        'undersampler__version': [1, 2, 3],
        'model__kernel': ['poly'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__degree': [i + 2 for i in range(4)],
        'model__coef0': [0, 0.1, 0.5, 1, 2, 10],
    },
    {
        'undersampler__version': [1, 2, 3],
        'model__kernel': ['rbf'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__gamma': [2 ** (i - 8) for i in range(17)],
    },
]
parameter_tuning = RandomizedSearchCV(
    svm_pipeline_undersampled,
    parameters,
    n_iter=50,
    scoring=diagnostic_odds_ratio_scorer,
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
svm_pipeline_undersampled = parameter_tuning.best_estimator_
display(svm_pipeline_undersampled)

### Oversampling
We will use SMOTENC oversampling.

In [ ]:
numeric_cols_count = len(numeric_cols)
total_cols_count = len(X_train.columns)
categorical_list = [i for i in range(numeric_cols_count, total_cols_count)]

In [ ]:
transformer_numeric = ColumnTransformer(
    [('num', StandardScaler(), numeric_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
transformer_categorical = ColumnTransformer(
    [('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_list)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
logistic_pipeline_oversampled = Pipeline(
    [
        ('transformer_numeric', transformer_numeric),
        ('oversampler', SMOTENC(categorical_list)),
        ('transformer_categorical', transformer_categorical),
        ('model', LogisticRegression(penalty=None)),
    ]
)

threshold_tuning = TunedThresholdClassifierCV(
    logistic_pipeline_oversampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
logistic_pipeline_oversampled = threshold_tuning.estimator_
logistic_threshold_oversampled = threshold_tuning.best_threshold_
display(logistic_pipeline_oversampled)
print('Best threshold:', logistic_threshold_oversampled)

In [ ]:
transformer_numeric = ColumnTransformer(
    [('num', StandardScaler(), numeric_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
transformer_categorical = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_list)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
forest_pipeline_oversampled = Pipeline(
    [
        ('transformer_numeric', transformer_numeric),
        ('oversampler', SMOTENC(categorical_list)),
        ('transformer_categorical', transformer_categorical),
        ('model', RandomForestClassifier()),
    ]
)

parameters = {
    'model__n_estimators': [100, 150, 200, 250],
    'model__criterion': ['gini', 'entropy'],
    'model__max_features': [0.1, 0.2, 0.3, 0.4],
    'model__max_depth': [2, 3, 5],
}
parameter_tuning = RandomizedSearchCV(
    forest_pipeline_oversampled,
    parameters,
    n_iter=50,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
forest_pipeline_oversampled = parameter_tuning.best_estimator_
display(forest_pipeline_oversampled)

threshold_tuning = TunedThresholdClassifierCV(
    forest_pipeline_oversampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
forest_threshold_oversampled = threshold_tuning.best_threshold_
print('Best threshold:', forest_threshold_oversampled)

In [ ]:
transformer_numeric = ColumnTransformer(
    [('num', StandardScaler(), numeric_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
transformer_categorical = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_list)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

boost_pipeline_oversampled = Pipeline(
    [
        ('transformer_numeric', transformer_numeric),
        ('oversampler', SMOTENC(categorical_list)),
        ('transformer_categorical', transformer_categorical),
        ('model', AdaBoostClassifier(algorithm='SAMME')),
    ]
)

parameters = {
    'model__n_estimators': [5, 10, 15, 20],
    'model__learning_rate': [0.4, 0.8, 1.2, 1.6, 2],
    'model__estimator': [
        DecisionTreeClassifier(
            max_depth=1,
            criterion=crit)
        for crit in ['gini', 'entropy']
    ],
}
parameter_tuning = GridSearchCV(
    boost_pipeline_oversampled,
    parameters,
    scoring='average_precision',
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
boost_pipeline_oversampled = parameter_tuning.best_estimator_
display(boost_pipeline_oversampled)

threshold_tuning = TunedThresholdClassifierCV(
    boost_pipeline_oversampled,
    scoring=diagnostic_odds_ratio_scorer,
    thresholds=valid_thresholds,
    cv=10,
)
threshold_tuning.fit(X_train, y_train)
boost_threshold_oversampled = threshold_tuning.best_threshold_
print('Best threshold:', boost_threshold_oversampled)

In [ ]:
transformer_numeric = ColumnTransformer(
    [('num', StandardScaler(), numeric_cols)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)
transformer_categorical = ColumnTransformer(
    [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_list)],
    remainder='passthrough',
    force_int_remainder_cols=False,
)

svm_pipeline_oversampled = Pipeline(
    [
        ('transformer_numeric', transformer_numeric),
        ('oversampler', SMOTENC(categorical_list)),
        ('transformer_categorical', transformer_categorical),
        ('model', SVC()),
    ]
)

parameters = [
    {
        'model__kernel': ['linear'],
        'model__C': [2 ** (i - 9) for i in range(19)],
    },
    {
        'model__kernel': ['poly'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__degree': [i + 2 for i in range(4)],
        'model__coef0': [0, 0.1, 0.5, 1, 2, 10],
    },
    {
        'model__kernel': ['rbf'],
        'model__C': [2 ** (i - 9) for i in range(10)],
        'model__gamma': [2 ** (i - 8) for i in range(17)],
    },
]
parameter_tuning = RandomizedSearchCV(
    svm_pipeline_oversampled,
    parameters,
    n_iter=50,
    scoring=diagnostic_odds_ratio_scorer,
    cv=10,
)
parameter_tuning.fit(X_train, y_train)
svm_pipeline_oversampled = parameter_tuning.best_estimator_
display(svm_pipeline_oversampled)

### Summary

In [ ]:
summary_data_test = pd.DataFrame(
    columns=['regular', 'weighted', 'undersampled', 'oversampled']
)
summary_data_test.loc['logistic_regression', 'regular'] = diagnostic_odds_ratio(
    y_test,
    logistic_pipeline.predict_proba(X_test)[:, 1]
    >= logistic_threshold,
)
summary_data_test.loc['logistic_regression', 'weighted'] = diagnostic_odds_ratio(
    y_test,
    logistic_pipeline_weighted.predict_proba(X_test)[:, 1]
    >= logistic_threshold_weighted,
)
summary_data_test.loc['logistic_regression', 'undersampled'] = diagnostic_odds_ratio(
    y_test,
    logistic_pipeline_undersampled.predict_proba(X_test)[:, 1]
    >= logistic_threshold_undersampled, 
)
summary_data_test.loc['logistic_regression', 'oversampled'] = diagnostic_odds_ratio(
    y_test,
    logistic_pipeline_oversampled.predict_proba(X_test)[:, 1]
    >= logistic_threshold_oversampled, 
)
summary_data_test.loc['random_forest', 'regular'] = diagnostic_odds_ratio(
    y_test,
    forest_pipeline.predict_proba(X_test)[:, 1]
    >= forest_threshold, 
)
summary_data_test.loc['random_forest', 'weighted'] = diagnostic_odds_ratio(
    y_test,
    forest_pipeline_weighted.predict_proba(X_test)[:, 1]
    >= forest_threshold_weighted, 
)
summary_data_test.loc['random_forest', 'undersampled'] = diagnostic_odds_ratio(
    y_test,
    forest_pipeline_undersampled.predict_proba(X_test)[:, 1]
    >= forest_threshold_undersampled, 
)
summary_data_test.loc['random_forest', 'oversampled'] = diagnostic_odds_ratio(
    y_test,
    forest_pipeline_oversampled.predict_proba(X_test)[:, 1]
    >= forest_threshold_oversampled, 
)
summary_data_test.loc['boosting', 'regular'] = diagnostic_odds_ratio(
    y_test,
    boost_pipeline.predict_proba(X_test)[:, 1]
    >= boost_threshold, 
)
summary_data_test.loc['boosting', 'weighted'] = diagnostic_odds_ratio(
    y_test,
    boost_pipeline_weighted.predict_proba(X_test)[:, 1]
    >= boost_threshold_weighted, 
)
summary_data_test.loc['boosting', 'undersampled'] = diagnostic_odds_ratio(
    y_test,
    boost_pipeline_undersampled.predict_proba(X_test)[:, 1]
    >= boost_threshold_undersampled,
)
summary_data_test.loc['boosting', 'oversampled'] = diagnostic_odds_ratio(
    y_test,
    boost_pipeline_oversampled.predict_proba(X_test)[:, 1]
    >= boost_threshold_oversampled,
)
summary_data_test.loc['svm', 'regular'] = diagnostic_odds_ratio(
    y_test,
    svm_pipeline_weighted.predict(X_test),
)
summary_data_test.loc['svm', 'weighted'] = diagnostic_odds_ratio(
    y_test,
    svm_pipeline.predict(X_test),
)
summary_data_test.loc['svm', 'undersampled'] = diagnostic_odds_ratio(
    y_test,
    svm_pipeline_undersampled.predict(X_test),
)
summary_data_test.loc['svm', 'oversampled'] = diagnostic_odds_ratio(
    y_test,
    svm_pipeline_oversampled.predict(X_test),
)
summary_data_test

In [ ]:
summary_data_train = pd.DataFrame(
    columns=['regular', 'weighted', 'undersampled', 'oversampled']
)
summary_data_train.loc['logistic_regression', 'regular'] = diagnostic_odds_ratio(
    y_train,
    logistic_pipeline.predict_proba(X_train)[:, 1]
    >= logistic_threshold,
)
summary_data_train.loc['logistic_regression', 'weighted'] = diagnostic_odds_ratio(
    y_train,
    logistic_pipeline_weighted.predict_proba(X_train)[:, 1]
    >= logistic_threshold_weighted,
)
summary_data_train.loc['logistic_regression', 'undersampled'] = diagnostic_odds_ratio(
    y_train,
    logistic_pipeline_undersampled.predict_proba(X_train)[:, 1]
    >= logistic_threshold_undersampled,
)
summary_data_train.loc['logistic_regression', 'oversampled'] = diagnostic_odds_ratio(
    y_train,
    logistic_pipeline_oversampled.predict_proba(X_train)[:, 1]
    >= logistic_threshold_oversampled,
)
summary_data_train.loc['random_forest', 'regular'] = diagnostic_odds_ratio(
    y_train,
    forest_pipeline.predict_proba(X_train)[:, 1]
    >= forest_threshold,
)
summary_data_train.loc['random_forest', 'weighted'] = diagnostic_odds_ratio(
    y_train,
    forest_pipeline_weighted.predict_proba(X_train)[:, 1]
    >= forest_threshold_weighted,
)
summary_data_train.loc['random_forest', 'undersampled'] = diagnostic_odds_ratio(
    y_train,
    forest_pipeline_undersampled.predict_proba(X_train)[:, 1]
    >= forest_threshold_undersampled,
)
summary_data_train.loc['random_forest', 'oversampled'] = diagnostic_odds_ratio(
    y_train,
    forest_pipeline_oversampled.predict_proba(X_train)[:, 1]
    >= forest_threshold_oversampled,
)
summary_data_train.loc['boosting', 'regular'] = diagnostic_odds_ratio(
    y_train,
    boost_pipeline.predict_proba(X_train)[:, 1]
    >= boost_threshold,
)
summary_data_train.loc['boosting', 'weighted'] = diagnostic_odds_ratio(
    y_train,
    boost_pipeline_weighted.predict_proba(X_train)[:, 1]
    >= boost_threshold_weighted,
)
summary_data_train.loc['boosting', 'undersampled'] = diagnostic_odds_ratio(
    y_train,
    boost_pipeline_undersampled.predict_proba(X_train)[:, 1]
    >= boost_threshold_undersampled,
)
summary_data_train.loc['boosting', 'oversampled'] = diagnostic_odds_ratio(
    y_train,
    boost_pipeline_oversampled.predict_proba(X_train)[:, 1]
    >= boost_threshold_oversampled,
)
summary_data_train.loc['svm', 'regular'] = diagnostic_odds_ratio(
    y_train,
    svm_pipeline.predict(X_train),
)
summary_data_train.loc['svm', 'weighted'] = diagnostic_odds_ratio(
    y_train,
    svm_pipeline_weighted.predict(X_train),
)
summary_data_train.loc['svm', 'undersampled'] = diagnostic_odds_ratio(
    y_train,
    svm_pipeline_undersampled.predict(X_train),
)
summary_data_train.loc['svm', 'oversampled'] = diagnostic_odds_ratio(
    y_train,
    svm_pipeline_oversampled.predict(X_train),
)
summary_data_train

It seems that random forest without any modifications is the way to go.

### Best Model

In [ ]:
tn, fp, fn, tp = \
confusion_matrix(
    y_test,
    forest_pipeline.predict_proba(X_test)[:, 1] >= forest_threshold,
).ravel()
print(
    'Accuracy:', (tp + tn) / (tp + fp + tn + fn),
    'Sensitivity:', tp / (tp + fn),
    'Specificity:', tn / (tn + fp),
    'Precision:', tp / (tp + fp),
)

In [ ]:
feature_importance = permutation_importance(
    forest_pipeline,
    X_test,
    y_test,
    n_repeats=1000,
)
importance_series = pd.Series(
    feature_importance.importances_mean,
    index=X_test.columns,
)

fig, ax = plt.subplots()
importance_series.plot.bar(yerr=feature_importance.importances_std, ax=ax)
ax.set_title('Feature importances using MDI')
ax.set_ylabel('Mean decrease in impurity')
fig.tight_layout()

It seems that only *Sex*, *FastingBS*, *Oldpeak* and *ST_Slope* may be considered important. This variables will be focused on in model interpretation.

### Model Interpretation

In [ ]:
observation = X_test.iloc[[100]]
display(observation)

#### Ceteris Paribus (CP) Profiles

In [ ]:
explainer = dx.Explainer(
    forest_pipeline,
    X_test,
    y_test,
    label='Random Forest',
    verbose=0,
)
pcp = explainer.predict_profile(observation)
pcp.plot()
pcp.plot(variable_type='categorical')

#### Partial Dependence (PD) Profile

In [ ]:
pdp = explainer.model_profile()
pdp.plot()
pdp.plot(geom='profiles')

In [ ]:
pdp = explainer.model_profile(variable_type='categorical')
pdp.plot()

In [ ]:
pdp = explainer.model_profile(groups='Sex', variables=['Oldpeak'])
pdp.plot(geom='profiles')
pdp = explainer.model_profile(groups='FastingBS', variables=['Oldpeak'])
pdp.plot(geom='profiles')
pdp = explainer.model_profile(groups='ST_Slope', variables=['Oldpeak'])
pdp.plot(geom='profiles')

#### Break-down Plots

In [ ]:
bd = explainer.predict_parts(
    observation,
    type='break_down_interactions',
    order=np.array(['Oldpeak', 'Sex', 'FastingBS', 'ST_Slope']),
)
bd.plot()
bd = explainer.predict_parts(
    observation,
    type='break_down_interactions',
    order=np.array(['Sex', 'FastingBS', 'ST_Slope', 'Oldpeak']),
)
bd.plot()
bd = explainer.predict_parts(
    observation,
    type='break_down_interactions',
    order=np.array(['Oldpeak', 'ST_Slope', 'Sex', 'FastingBS']),
)
bd.plot()
bd = explainer.predict_parts(
    observation,
    type='break_down_interactions',
    order=np.array(['ST_Slope', 'Sex', 'FastingBS', 'Oldpeak']),
)
bd.plot()

#### SHAP Values

In [ ]:
shap = explainer.predict_parts(observation, type='shap')
shap.plot()